In [ ]:
# Cell 1: Imports, hyperparams & data prep (real + synth), normalization, poly‐feature cache
import random, re, math
import numpy as np, pandas as pd
import sympy as sp
import tensorflow as tf, mpmath as mp
from functools import lru_cache
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import (
    Input, TextVectorization, Embedding, LSTM,
    Dense, Concatenate, Lambda
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TerminateOnNaN

x = sp.symbols('x')
SEQ_LEN, BATCH, LR, EPOCHS, INT_TOL = 64, 64, 1e-4, 20, 1e-6

# load real data
df_real = pd.read_csv("functions2.csv", header=None,
    names=["function","lower","upper","true_raw"])
df_real["lower"]    = pd.to_numeric(df_real["lower"], errors="coerce")
df_real["upper"]    = pd.to_numeric(df_real["upper"], errors="coerce")
df_real["true_raw"] = pd.to_numeric(df_real["true_raw"], errors="coerce")
df_real.dropna(subset=["lower","upper","true_raw"], inplace=True)
df_real.reset_index(drop=True, inplace=True)

# synthetic polys
@lru_cache(None)
def make_random_poly(d):
    cs = [random.uniform(-5,5) for _ in range(d+1)]
    return sum(c*x**i for i,c in enumerate(cs))

@lru_cache(None)
def integrate_sympy(s,a,b):
    e = sp.sympify(s.replace('^','**').replace('ln(','log('))
    return float(sp.integrate(e,(x,a,b)))

rows = []
for _ in range(5000):
    d = random.randint(1,5)
    e = make_random_poly(d)
    a, b = random.uniform(-3,0), random.uniform(0,3)
    s = integrate_sympy(str(e).replace('**','^'),a,b)
    rows.append({
        "function": str(e).replace('**','^'),
        "lower":    a,
        "upper":    b,
        "true_raw": s
    })
df_synth = pd.DataFrame(rows)

# combine, shuffle & norm
df = pd.concat([df_real,df_synth], axis=0).sample(frac=1,random_state=42)
y_mean, y_std = df["true_raw"].mean(), df["true_raw"].std()
df["y_norm"]  = (df["true_raw"] - y_mean)/y_std
l_mean, l_std = df["lower"].mean(), df["lower"].std()
u_mean, u_std = df["upper"].mean(), df["upper"].std()
df["lower_n"] = (df["lower"] - l_mean)/l_std
df["upper_n"] = (df["upper"] - u_mean)/u_std

@lru_cache(None)
def poly_feats_cached(s):
    try:
        e = sp.sympify(s.replace('^','**').replace('ln(','log('))
        p = sp.Poly(e, x)
        return tuple(float(p.coeff_monomial(x**i)) for i in range(6))
    except:
        return (0.0,)*6

df["poly_feats"] = df["function"].apply(poly_feats_cached)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print("Total samples:", len(df))


2025-06-12 17:03:40.346389: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Cell 2: Text‐vectorizer + dataset builder (handles '^'→'**', 'ln('→'log(')
vectorizer = TextVectorization(
    output_mode="int",
    output_sequence_length=SEQ_LEN,
    standardize=lambda s:
      tf.strings.regex_replace(
        tf.strings.regex_replace(tf.strings.lower(s),
                                 r"\^","**"),
                                 r"ln\(","log("),
    split="character"
)
vectorizer.adapt(train_df["function"].values)

def make_ds(ddf, shuffle=True):
    X = {
      "func_input":  ddf["function"].values,
      "lower_input": ddf["lower_n"].values.astype(np.float32),
      "upper_input": ddf["upper_n"].values.astype(np.float32),
      "poly_feats":  np.stack(ddf["poly_feats"].values).astype(np.float32),
    }
    y = ddf["y_norm"].values.astype(np.float32)
    ds = tf.data.Dataset.from_tensor_slices((X,y))
    if shuffle: ds = ds.shuffle(len(ddf))
    return ds.batch(BATCH).cache().prefetch(tf.data.AUTOTUNE)

train_ds = make_ds(train_df, shuffle=True)
val_ds   = make_ds(val_df,   shuffle=False)


In [ ]:
# Cell 3: Build 3‐branch model (chars → LSTM, bounds, poly‐feats → merge)
f_in = Input(shape=(), dtype=tf.string, name="func_input")
toks = vectorizer(f_in)
toks = Lambda(lambda t: tf.cast(t,tf.int32))(toks)
c = Embedding(vectorizer.vocabulary_size(),32,mask_zero=True)(toks)
c = LSTM(32)(c)

l_in = Input((1,),dtype=tf.float32,name="lower_input")
u_in = Input((1,),dtype=tf.float32,name="upper_input")
b   = Concatenate()([l_in,u_in])
b   = Dense(16,activation="relu")(b)
b   = Dense(8, activation="relu")(b)

p_in = Input((6,),dtype=tf.float32,name="poly_feats")
p   = Dense(32,activation="relu")(p_in)
p   = Dense(16,activation="relu")(p)

m = Concatenate()([c,b,p])
m = Dense(64,activation="relu")(m)
m = Dense(32,activation="relu")(m)
out = Dense(1,activation="linear",name="pred")(m)

model = Model([f_in,l_in,u_in,p_in],out)
model.compile(Adam(LR,clipnorm=1.0), "huber", metrics=["mae"])
model.summary()


In [ ]:
# Cell 4: Train & evaluate
model.fit(train_ds, validation_data=val_ds,
          epochs=EPOCHS, callbacks=[TerminateOnNaN()])
loss, mae = model.evaluate(val_ds)
print(f"Val Huber={loss:.4f}, MAE(norm)={mae:.4f}")


Total samples: 207252


In [ ]:
# Cell 5: Safe inference → NN, then Sympy/mpmath, then trapezoid
def safe_integral(func_str, a, b, tol=INT_TOL):
    expr_in = func_str.replace('^','**').replace('ln(','log(')
    ln_n = (a - l_mean)/l_std
    un_n = (b - u_mean)/u_std
    pf   = np.array(poly_feats_cached(func_str),dtype=np.float32)[None]
    # NN
    fn = np.array([func_str],dtype=object)
    rn = model.predict([fn,
                        [[ln_n]],
                        [[un_n]],
                        pf],verbose=0)[0,0]
    pn = rn*y_std + y_mean
    if not np.isnan(pn) and abs(pn-round(pn))<tol:
        return round(pn)
    # exact
    try:
        e = sp.sympify(expr_in)
        return float(sp.integrate(e,(x,a,b)))
    except:
        pass
    # numeric
    try:
        e = sp.sympify(expr_in,convert_xor=True)
        fmp = sp.lambdify(x,e,'mpmath')
        return float(mp.quad(fmp,[a,b]))
    except:
        pass
    # fallback
    safe = re.sub(r'(?<=\d)(?=[A-Za-z\(])','*',expr_in)
    f = lambda v: eval(safe,{"x":v,**math.__dict__})
    xs = np.linspace(a,b,2000)
    ys = [f(v) for v in xs]
    return float(np.trapz(ys,xs))


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [ ]:
# Cell 6: Interactive prompt (fractions in func, ln(), etc. all work)
def predict_interactive():
    f = input("Function (e.g. x^2/(x^3-1)+ln(x)): ")
    a = float(input("Lower bound: "))
    b = float(input("Upper bound: "))
    print("Result ≃", safe_integral(f,a,b))

predict_interactive()
